<a href="https://colab.research.google.com/github/RaianaSeixas/0_paper_one_revisited/blob/main/Frannk_2000_Schwefel_223_D_10_0_5_0_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np
import pandas as pd
import time

In [18]:
#!git clone https://github.com/ucfilho/Raianars_July_2020 #clonar do Github
#%cd Raianars_July_2020
############################### Importar códigos
!git clone https://github.com/ucfilho/optim_raianars_Insight
%cd optim_raianars_Insight

Cloning into 'optim_raianars_Insight'...
remote: Enumerating objects: 375, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 375 (delta 35), reused 0 (delta 0), pack-reused 314
Receiving objects: 100% (375/375), 327.03 KiB | 12.58 MiB/s, done.
Resolving deltas: 100% (216/216), done.
/content/optim_raianars_Insight/ANN_read_matrix/optim_raianars_Insight/ANN_read_matrix/optim_raianars_Insight


In [19]:
import Go2Ann
import de_soma_Insight
import de_bounds_matriz
from AvaliaX import AvaliaX
from GeraIndices import GeraIndices

In [20]:
import Function
import Go2MinMax

In [21]:
ITE=2000 #10 # PONTOS quantas vezes vai alteral NPAR 
NPAR=100 # NUMBER OF PARTICLES
dim=10
RUNS=50
fchoice='Schwefel_223'  #MS
Fun=Function.Schwefel_223

MIN,MAX=Go2MinMax.Intervalo(dim,fchoice)
bounds = [(0,0)]*dim

for i in range(dim):
  bounds[i] = (MIN[i],MAX[i])

In [22]:
############################### INPUT INFORMATION (FRANNK)
Fc=0.1 # fator de correcao parametros da ANN
its=1 #10 # quantas vezes vai usar o NPAR antes de alterar de novo
mut=0.5
crossp=0.5
Fi=mut;CRi=crossp;# para impressão
PAR=len(MAX) #NUM DE PARAMETROS A SER OTIMIZADOS

In [23]:
!git clone https://github.com/ucfilho/ANN_read_matrix #clonar do Github
%cd ANN_read_matrix

Cloning into 'ANN_read_matrix'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 20 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (20/20), done.
/content/optim_raianars_Insight/ANN_read_matrix/optim_raianars_Insight/ANN_read_matrix/optim_raianars_Insight/ANN_read_matrix


In [24]:
ANN_F=pd.read_csv('ANN_INSIGHT_Valor_F.csv')
ANN_CR=pd.read_csv('ANN_INSIGHT_CR.csv')
syn0_F,syn1_F,X_max_F,X_min_F=Go2Ann.GetMatriz(ANN_F)
syn0_CR,syn1_CR,X_max_CR,X_min_CR=Go2Ann.GetMatriz(ANN_CR)
fields = ['DI RELATIVO','FRAC Its','Fo','CRo']

#TOTAL=7*dim*its*ITE
TOTAL=10*dim*its*ITE
TOTAL=int(TOTAL/2)   # CONFERIR SE ESTE EH O MELHOR LUGAR

dimensions=len(MAX)
start_time = time.time() #INICIALIZA CONTAGEM DO TEMPO

MAX_P=np.zeros(ITE) # ATENCAO para variar F e Cr MAX_P passa a ser 3 vezes maior
MIN_P=np.zeros(ITE) # ATENCAO para variar F e Cr MIN_P passa a ser 3 vezes maior

for i in range(dim):
  MAX_P[i]=MAX[i] 
  MIN_P[i]=MIN[i]

rodadas=1
for i in range(rodadas):
  SOMA=0
  pop = np.random.rand(NPAR, dimensions)
  diff=np.zeros(PAR)
  for ii in range(PAR):
    diff[ii] = MAX[ii]-MIN[ii]
  X = MIN + pop * diff
    
  PONTOS=ITE # DIVISOES DA POPULACAO
start_time = time.time()
BEST_AGENTS=[]
MAT_stat=[]
for i in range(RUNS):
    MAT=[] 
    mut=Fi
    crossp=CRi
    SOMA=0
    SOMAf=0
    start_ite = time.time()
    delta_time=(start_ite-start_time)/60
    #print('ite=',i,'total=50','time=',delta_time)
    pop = np.random.rand(NPAR, dim)
    diff=np.zeros(PAR)
    for ii in range(PAR):
      diff[ii] = MAX[ii]-MIN[ii]
    X = MIN + pop * diff
    Xf = MIN + pop * diff
    
    DIo=1e99
  
    CRo=crossp;Fo=mut;
    Fitness = np.asarray([Fun(ind) for ind in X])

    XY,BEST_XY,BESTo,FOBESTo=AvaliaX(X,Fitness)
    nrow,ncol=X.shape

    QUANT=17;MAT_INDo=np.zeros((1,QUANT)); # quantos indices esta fazendo
    MAT_INDo[0,5]=nrow  # VALOR NP
    MAT_INDo[0,6]=Fo    # VALOR Fo   
    MAT_INDo[0,7]=CRo   # VALOR CRo
  
    MAT_INDo=GeraIndices(X,BESTo,FOBESTo,DIo,MAT_INDo,SOMA,TOTAL,syn0_F,
                        syn1_F,X_max_F,X_min_F,syn0_CR,syn1_CR,X_max_CR,
                        X_min_CR,Fc,fields,Fun)
  
    DIo=np.copy(MAT_INDo[0,0])
    MAT_INDo[0,15]=Fo    # VALOR Fo   
    MAT_INDo[0,16]=CRo   # VALOR CRo
    #ind_crossp=ITE-1

    
    for k in range(ITE):
      X,BEST,FOBEST,XY,BEST_XY,SOMA= de_soma_Insight.de(MAX,MIN, mut, crossp,NPAR,its,Fun,X,SOMA,TOTAL)
      #Xf,BESTf,FOBESTf,XYf,BEST_XYf,SOMAf= de_soma_Insight.de(MAX,MIN, Fi, CRi, NPAR,its,Fun,Xf,SOMAf,TOTAL)

      
      MAT_IND=GeraIndices(X,BESTo,FOBESTo,DIo,MAT_INDo,SOMA,TOTAL,syn0_F,
                          syn1_F,X_max_F,X_min_F,syn0_CR,syn1_CR,X_max_CR,
                          X_min_CR,Fc,fields,Fun)
    
      BESTo=np.copy(BEST);FOBESTo=np.copy(FOBEST);
      MAT.append(MAT_IND.ravel())
    
      MAT_INDo[0,6]=MAT_IND[0,15]   # VALOR Fo   
      MAT_INDo[0,7]=MAT_IND[0,16]   # VALOR CRo
      mut=MAT_IND[0,15]
      crossp=MAT_IND[0,16]

      Fitness = np.asarray([Fun(ind) for ind in X])
      #Fitnessf = np.asarray([Fun(ind) for ind in Xf])
      Maior=np.argmax(Fitness)
      #Maiorf=np.argmax(Fitnessf)
      Menor=np.argmin(Fitness)
      #Menorf=np.argmin(Fitnessf)

      #X[Maior,:]=np.copy(Xf[Menorf,:])
      #Xf[Maiorf,:]=np.copy(X[Menor,:])
    print('i=',i,'time=',(time.time()-start_time)/60, 'FO=',FOBEST)  
      
    BEST_AGENTS.append(BEST_XY)
    MAT_stat.append(MAT)
    #print("GlobalParam e Solucao:", BEST,"Funcao=",FOBEST)
    
############################## Cálculo dos Parâmetros de Avaliação   
TIME = (time.time() - start_time)/RUNS
BEST_AGENTS=np.array(BEST_AGENTS)
BEST_AGENTS_ST= BEST_AGENTS[BEST_AGENTS[:,-1].argsort()]
BEST=BEST_AGENTS_ST[0]
WORSE=BEST_AGENTS_ST[-1]
MEAN=np.mean(BEST_AGENTS,axis=0)
MAXFO=BEST_AGENTS[:,-1].max()
MINFO=BEST_AGENTS[:,-1].min()
STD=BEST_AGENTS[:,-1].std()

nome='_ANN_Div3x4_Fo-'+str(Fi)+'_CRo-'+str(CRi)+'_D-'+str(dim)+'_Interv-'+str(ITE)
print('DE_POP_VAR_F_CR','TOTAL=',TOTAL,'Iterações=',ITE*its,'Dim=', PAR,"Min/Max=",MIN[0],"/",MAX[0],'_',Fun.__name__)
print(nome, '\n')
print('MÉDIA: ',MEAN[-1])
print('MIN __:',MINFO)
print('MAX__: ',MAXFO)
print('STD __:',STD)
print("Tempo Médio= %.3f segundos ---" % (TIME))
print(RUNS,'Rodadas') 
print()
print('BEST: ',BEST)

i= 0 time= 0.8607945680618286 FO= 5.843765547506923e-234
i= 1 time= 1.709054676691691 FO= 3.187107820200926e-233
i= 2 time= 2.5590498288472494 FO= 2.6732051988524775e-234
i= 3 time= 3.422051787376404 FO= 1.3159493781595008e-235
i= 4 time= 4.292864386240641 FO= 9.488415286011317e-233
i= 5 time= 5.155522890885671 FO= 3.6053004817883764e-232
i= 6 time= 6.035007389386495 FO= 1.1285303682961666e-239
i= 7 time= 6.905688746770223 FO= 6.972705688080188e-237
i= 8 time= 7.77069456577301 FO= 3.3948847349705595e-236
i= 9 time= 8.62990673383077 FO= 1.92722939402023e-232
i= 10 time= 9.535685058434805 FO= 6.0296398797622245e-233
i= 11 time= 10.39472103913625 FO= 1.9400373841446674e-232
i= 12 time= 11.250107097625733 FO= 1.1728960431920845e-235
i= 13 time= 12.104205401738485 FO= 3.922848658155826e-233
i= 14 time= 12.966786638895671 FO= 1.5877546250645342e-231
i= 15 time= 13.820328652858734 FO= 3.4232440705704505e-232
i= 16 time= 14.668879330158234 FO= 3.9770902194445526e-234
i= 17 time= 15.50957453250